In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd

# Para visualización de datos
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt

# Para cargar los datos
# -----------------------------------------------------------------------
import pickle
import sys
import os
from time import sleep
sys.path.append(os.path.abspath('../'))
# Para sistemas de recomendacion basados en cotenido
# -----------------------------------------------------------------------
from scipy.spatial.distance import pdist , squareform   # Para calcular la distancia entre puntos 

# Importamos nuestras funciones
# -----------------------------------------------------------------------
from src import soporte_sistemas_recomendacion as sr
from src import soporte_tablas as st

# Configuración de pandas
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)

In [2]:
conexion = st.conexion("IMDB")
cursor = conexion.cursor()  

In [3]:
cursor.execute('SELECT * FROM "pelicula" pe, "artistas_detalles" ad  where pe."Id"= ad."Id" ;')  
datos_ejercicios = cursor.fetchall() 
columnas_ejercicios = [desc[0] for desc in cursor.description]  
df_notas = pd.DataFrame(datos_ejercicios, columns=columnas_ejercicios) 


In [4]:
metadata_final = df_notas[['Tipo','Titulo','generos','cali_imdb']]

In [5]:
# seleccionamos las columnas de interés y renombramos el nombre de las columnas
df_collab = metadata_final[['Tipo','Titulo','generos','cali_imdb']]

df_collab.head()

,Tipo,Titulo,generos,cali_imdb
0,Movie,Las Travesuras de Super Chido,Action,7.7
1,Movie,Velai Kidaichiruchu,Action,5.2
2,Movie,Sor Batalla,Action,6.6
3,Movie,Azaad Desh Ke Gulam,Action,5.3
4,Movie,Suo ming fei dao,Action,NaN


In [6]:
df_collab = df_collab.dropna()
df_collab

,Tipo,Titulo,generos,cali_imdb
0,Movie,Las Travesuras de Super Chido,Action,7.7
1,Movie,Velai Kidaichiruchu,Action,5.2
2,Movie,Sor Batalla,Action,6.6
3,Movie,Azaad Desh Ke Gulam,Action,5.3
6,Movie,Pitong Gamol,Action,5.9
...,...,...,...,...
1443,Short,The Turn Ahead,Mystery,8.8
1444,Movie,Maksym Osa,Mystery,6.0
1445,Short,Agents of Change: Project Polymer,Mystery,8.5
1446,Movie,Khitrovka. The Sign of Four,Mystery,5.0


In [7]:
# pivitamos la tabla
userRatings = df_collab.pivot_table(index=['generos'],columns=['Titulo'],values='cali_imdb').fillna(0)
userRatings.head()


Titulo,'Ne günstige Gelegenheit,12 Feet Deep,12/12/2012,13 segundos,14 Days in Paradise,18 anni tra una settimana,25Gs,3 Chinesen mit dem Kontrabass,3 Days of Darkness,3:15 AM,5,6079_Zn,7,A Beautiful Mess,A Good Dream,A Haunting at The Octagon Mansion,A Little Bit of Lipstick,A Matter of Degrees,A Rainy Day,A Shameless Revenge,A Smile Like Yours,A Taste,A Vigilante,A Waiter Tomorrow,A kis utazás,A praga,Aa Intlo,Ab... Bas!,About A Short Film,Accused at 17,Achanak,Achoura,Adam & Eva,Afflatus,Afghan Knights,After the Third Bell,Aftermath: A Test of Love,Agatha,Agents of Change: Project Polymer,Aibô: Gekijô-ban IV,Akala mo...,Ako si Ninoy,All Flowers in Time,All My Life,All the Fists in the World Won't Save You Now,Almost Heaven,Alternate,Alyti,Am I,American Exorcist,American Short,Among Thieves,Amor que mata,An Idle Mind Is the Devil's Playground,An Unexpected Love,Anakiseuteu Anarchists,Angaaray,Annabelle: Creation,Annabellum: The Curse of Salem,Anomie,Anonymous Vol. 1 - The Dreamland Adventures the Mysteries of Atlantis,Anti Cupido,Any Way the Wind Blows,Apocalypse Road,Apostle,Apparition,Armugan,Aryan: Unbreakable,Asbury Park,Askin Gönül Yazisi,Astro,Aswang: A Journey Into Myth,Attack of the Adult Babies,Aux,Awaken,Azaad Desh Ke Gulam,BIR,Bacao,Bakom Snart är det lördag igen,Ban khang winyan,Bandwagon,Bangkok Dangerous,Bank Robber,Bannerghatta,Batman: Dead End,Bats in the Belfry,Bbuddah... Hoga Terra Baap,Be-seu-teu-sel-leo,Beat,Ben: The Movie,Bernard and Doris,Between the Trees,Bhoner: The Movie,Bigfoot,Birthday,Bit Players,Black Pond,Black Pumpkin,Blarghaaahrgarg,Blindsided,Blood Vow,Blue,Boarding School,Bob Roberts,Bobby,Body Bags,Bolse vita,Bonnie & Clyde vs. Dracula,Boogeyman Pop,"Boonie Bears, to the Rescue!",Boy Friend,Boy Meets Girl Stories #4: Geen antwoord,BoyTown,Boychick,Brightheart,Bun siu hai,Buster's Mal Heart,Buttf**ker,Caliche Run,Call Me Claus,Calling,Camp X-Ray: Ghosts of Guantánamo Bay,Captive Heart: The James Mink Story,Casablanca,Cassette,Caustic Zombies,Ce jour-là,Cerro Torre: Schrei aus Stein,Cesta peklem,Chaco,Charlie St. Cloud,Cheun,Cheuuat gaawn chim,Chi bi: Jue zhan tian xia,Children of the Grave 2,Chingaari,Chong qing ai qing gan jue,"Chot [Aaj Isko, Kal Tereko]",Christian Blake,Christmas Twister,Christmas on Division Street,Chuck Steel: Night of the Trampires,Churuli,Circles,Class Savage,Client Seduction,CobraGator,Color-Blinded,Como Nascem os Anjos,Company K,Cowboy,Creature,Criss Angel Mindfreak,Criss Angel Supernatural,Crna ovca u restoranu za zombije,Crumbs,Cul de sac,Cumbia callera,Curse of Mesopotamia,Daava,Daddy Cross,Dagaalty,Dai zek lo,Dante's View,Dark Heart,Darkness Reigns,Darkwood Manor,Dawna of the Dead,Day 66,De zaak Alzheimer,Dead Again,Dead Ahead,Dead Bodies Everywhere,Dead of Winter,Deadtime,Dear Father,"Death, Deceit & Destiny Aboard the Orient Express",Demons,Den Japanska Illusionen,Den sorte celle,Der Traum vom Süden,Der Wien-Krimi: Blind ermittelt - Tod an der Donau,Der Zauber des Regenbogens,Detektor,Devil's Pond,Devot,Dibbuk,Die Mediocren,Die Waschmaschine,Die andere Frau,Die: Lianhua,Diljale,Dinner with Friends,Direktøren for det hele,Distants,Distorted,Divar,Dok,Doli Saja Ke Rakhna,Dolphins,Don't Look Under the Bed,Don't Look in the Basement,Dontel papa,Doraemon: Nobita to fushigi kazetsukai,Doug's 1st Movie,Dr. Munshir Diary,Dream a Little Dream for Me,DristiBhram,Dubai,Duo ming jie chu,Easter Bunny Bloodbath,Eat Me,Eating Out,Eccezzziunale veramente - Capitolo secondo... me,Ecco fatto,Echoes of the Passed,Eddie,Ein Sack voll Geld,Ek: The Power of One,El ataque de los robots de nebulosa-5,El bar,El corazón de la tierra,El domini del sentits,El grito en el cielo,El que habita las alturas,El sol,El sueño de Ibiza,Elaan,Ellos Volvieron,Elsa y Fred,Empaná de pino,Encrucijada,End of the Line,Endoresu warutsu,Enemy Within,Enfants terribles,"English, August",Epoch: Evolution,Erakkorapu,Escape Room,Eskiya,Estrela Radiante,"Eternal Coma, 1994",Ex Abyssus,Exile,Exit 14,Exorsister,E

In [8]:
# calculamos la matriz de correlacion
matr_corre = userRatings.corr(method='pearson')
matr_corre.head()

Titulo,'Ne günstige Gelegenheit,12 Feet Deep,12/12/2012,13 segundos,14 Days in Paradise,18 anni tra una settimana,25Gs,3 Chinesen mit dem Kontrabass,3 Days of Darkness,3:15 AM,5,6079_Zn,7,A Beautiful Mess,A Good Dream,A Haunting at The Octagon Mansion,A Little Bit of Lipstick,A Matter of Degrees,A Rainy Day,A Shameless Revenge,A Smile Like Yours,A Taste,A Vigilante,A Waiter Tomorrow,A kis utazás,A praga,Aa Intlo,Ab... Bas!,About A Short Film,Accused at 17,Achanak,Achoura,Adam & Eva,Afflatus,Afghan Knights,After the Third Bell,Aftermath: A Test of Love,Agatha,Agents of Change: Project Polymer,Aibô: Gekijô-ban IV,Akala mo...,Ako si Ninoy,All Flowers in Time,All My Life,All the Fists in the World Won't Save You Now,Almost Heaven,Alternate,Alyti,Am I,American Exorcist,American Short,Among Thieves,Amor que mata,An Idle Mind Is the Devil's Playground,An Unexpected Love,Anakiseuteu Anarchists,Angaaray,Annabelle: Creation,Annabellum: The Curse of Salem,Anomie,Anonymous Vol. 1 - The Dreamland Adventures the Mysteries of Atlantis,Anti Cupido,Any Way the Wind Blows,Apocalypse Road,Apostle,Apparition,Armugan,Aryan: Unbreakable,Asbury Park,Askin Gönül Yazisi,Astro,Aswang: A Journey Into Myth,Attack of the Adult Babies,Aux,Awaken,Azaad Desh Ke Gulam,BIR,Bacao,Bakom Snart är det lördag igen,Ban khang winyan,Bandwagon,Bangkok Dangerous,Bank Robber,Bannerghatta,Batman: Dead End,Bats in the Belfry,Bbuddah... Hoga Terra Baap,Be-seu-teu-sel-leo,Beat,Ben: The Movie,Bernard and Doris,Between the Trees,Bhoner: The Movie,Bigfoot,Birthday,Bit Players,Black Pond,Black Pumpkin,Blarghaaahrgarg,Blindsided,Blood Vow,Blue,Boarding School,Bob Roberts,Bobby,Body Bags,Bolse vita,Bonnie & Clyde vs. Dracula,Boogeyman Pop,"Boonie Bears, to the Rescue!",Boy Friend,Boy Meets Girl Stories #4: Geen antwoord,BoyTown,Boychick,Brightheart,Bun siu hai,Buster's Mal Heart,Buttf**ker,Caliche Run,Call Me Claus,Calling,Camp X-Ray: Ghosts of Guantánamo Bay,Captive Heart: The James Mink Story,Casablanca,Cassette,Caustic Zombies,Ce jour-là,Cerro Torre: Schrei aus Stein,Cesta peklem,Chaco,Charlie St. Cloud,Cheun,Cheuuat gaawn chim,Chi bi: Jue zhan tian xia,Children of the Grave 2,Chingaari,Chong qing ai qing gan jue,"Chot [Aaj Isko, Kal Tereko]",Christian Blake,Christmas Twister,Christmas on Division Street,Chuck Steel: Night of the Trampires,Churuli,Circles,Class Savage,Client Seduction,CobraGator,Color-Blinded,Como Nascem os Anjos,Company K,Cowboy,Creature,Criss Angel Mindfreak,Criss Angel Supernatural,Crna ovca u restoranu za zombije,Crumbs,Cul de sac,Cumbia callera,Curse of Mesopotamia,Daava,Daddy Cross,Dagaalty,Dai zek lo,Dante's View,Dark Heart,Darkness Reigns,Darkwood Manor,Dawna of the Dead,Day 66,De zaak Alzheimer,Dead Again,Dead Ahead,Dead Bodies Everywhere,Dead of Winter,Deadtime,Dear Father,"Death, Deceit & Destiny Aboard the Orient Express",Demons,Den Japanska Illusionen,Den sorte celle,Der Traum vom Süden,Der Wien-Krimi: Blind ermittelt - Tod an der Donau,Der Zauber des Regenbogens,Detektor,Devil's Pond,Devot,Dibbuk,Die Mediocren,Die Waschmaschine,Die andere Frau,Die: Lianhua,Diljale,Dinner with Friends,Direktøren for det hele,Distants,Distorted,Divar,Dok,Doli Saja Ke Rakhna,Dolphins,Don't Look Under the Bed,Don't Look in the Basement,Dontel papa,Doraemon: Nobita to fushigi kazetsukai,Doug's 1st Movie,Dr. Munshir Diary,Dream a Little Dream for Me,DristiBhram,Dubai,Duo ming jie chu,Easter Bunny Bloodbath,Eat Me,Eating Out,Eccezzziunale veramente - Capitolo secondo... me,Ecco fatto,Echoes of the Passed,Eddie,Ein Sack voll Geld,Ek: The Power of One,El ataque de los robots de nebulosa-5,El bar,El corazón de la tierra,El domini del sentits,El grito en el cielo,El que habita las alturas,El sol,El sueño de Ibiza,Elaan,Ellos Volvieron,Elsa y Fred,Empaná de pino,Encrucijada,End of the Line,Endoresu warutsu,Enemy Within,Enfants terribles,"English, August",Epoch: Evolution,Erakkorapu,Escape Room,Eskiya,Estrela Radiante,"Eternal Coma, 1994",Ex Abyssus,Exile,Exit 14,Exorsister,E

In [9]:
nombre_movie = '13 segundos'
rating = 5

similar_ratings = matr_corre[nombre_movie]*(rating-2.5)
recommendacion = similar_ratings.sort_values(ascending=False)

# printemoas la recomendacion
print(recommendacion[1:11].sort_values(ascending=False))

Titulo
A Shameless Revenge               2.5
¡Atraco!                          2.5
A Beautiful Mess                  2.5
Troglodyte                        2.5
The Loner                         2.5
Annabellum: The Curse of Salem    2.5
The Dark                          2.5
The Keeping Hours                 2.5
Second Sight                      2.5
Remains                           2.5
Name: 13 segundos, dtype: float64
